# Segmenting and Clustering Neighborhoods in Toronto

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

1-Start by creating a new Notebook for this assignment.

2-Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe 

3-To create the dataframe:
* List The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.
* List Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
* List More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
* List If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
* List Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
* List In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

## PART 1 Create Data Frame

In [1]:
!pip install beautifulsoup4
!pip install lxml
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes 
from bs4 import BeautifulSoup as soup
import requests
import pandas as pd
import lxml
from pandas import DataFrame
import numpy as np

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



**Obtain the list of postal codes of Canada** 

In [2]:
ca_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
ca_list = requests.get(ca_url)
ca_list

<Response [200]>

**pulling data out of HTML and XML files with Beautiful Soup**

In [3]:
page=soup(ca_list.text,"html.parser")
#page

**Select the table from the HTML**

In [4]:
ca_table=page.table  #will select the <table class="wikitable sortable"> inside html code


In [5]:
#count the number of rows inside the table (ca_table), we need to look the HTML code <tr>
tr=ca_table.find_all('tr')
rows=len(tr)
rows



181

In [6]:
#we have 181 rows with the next format:
tr[0:5]

[<tr>
 <th>Postal Code
 </th>
 <th>Borough
 </th>
 <th>Neighborhood
 </th></tr>,
 <tr>
 <td>M1A
 </td>
 <td>Not assigned
 </td>
 <td>
 </td></tr>,
 <tr>
 <td>M2A
 </td>
 <td>Not assigned
 </td>
 <td>
 </td></tr>,
 <tr>
 <td>M3A
 </td>
 <td>North York
 </td>
 <td>Parkwoods
 </td></tr>,
 <tr>
 <td>M4A
 </td>
 <td>North York
 </td>
 <td>Victoria Village
 </td></tr>]

In [7]:
#Let's obtain the headers that are in the first line of code
headers=tr[0].text.split()
headers




['Postal', 'Code', 'Borough', 'Neighborhood']

In [8]:
#We need to extract each row of the table to extract the information into our dataframe.
#let's check the type of information stored for example in line 10
tr[10].text

'\nM1B\n\nScarborough\n\nMalvern, Rouge\n'

In [9]:
#we can split the line using text.split and indicated the line break code \n
tr[10].text.split('\n')

['', 'M1B', '', 'Scarborough', '', 'Malvern, Rouge', '']

In [10]:
#we have a list, and we can extract each information of it:ç
postalcode=tr[10].text.split('\n')[1]
borough=tr[10].text.split('\n')[3]
neighborhood=tr[10].text.split('\n')[5]

print("the postal code is: " + postalcode + " the borough is: " + borough + " and the nb is: " + neighborhood)

the postal code is: M1B the borough is: Scarborough and the nb is: Malvern, Rouge


In [11]:
#now we need to create a while loop to obtain in each row the values of postal code, borough and neghtborhood and store it in a dataframe using pandas
obtain_values =[]
n= 1
while n < rows:
    Postcode=tr[n].text.split('\n')[1]
    Borough=tr[n].text.split('\n')[3]
    Neighborhood=tr[n].text.split('\n')[5]
    obtain_values.append((Postcode, Borough,Neighborhood))
    n=n+1

ca_df=pd.DataFrame(obtain_values, columns=['PostalCode', 'Borough', 'Neighborhood'])
#note that we also merge the repited postalcodes, like M5A that we can check with: ca_df.head(5)
ca_df.head(5) 


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [12]:
ca_df.shape

(180, 3)

In [13]:
#let's check if is included the complete table checking the last 5 rows
ca_df.tail()

,PostalCode,Borough,Neighborhood
175,M5Z,Not assigned,
176,M6Z,Not assigned,
177,M7Z,Not assigned,
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."
179,M9Z,Not assigned,


In [14]:
#drop all cells in Borough and neighborhood colu& mn with not assigned statement
ca_df=ca_df[(ca_df.Borough != 'Not assigned') | (ca_df.Neighborhood == 'Not assigned')]
ca_df.reset_index(drop=True, inplace=True) #reset the index after drop
ca_df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [15]:
ca_df.shape

(103, 3)

## END OF PART 1

## PART 2 GET LATITUDE AND LONGITUDE FOR EACH POSTAL CODE

Geocoder did not worked. So we will read the excel file with the latitudes and longitudes for each Postal Code



In [16]:
#read excel file and create a new dataframe
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')
geo_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [17]:
#the column Postal Code must be renamed as PostalCode if we want to merge our previous dataframe with the new one.
geo_df.rename(columns={'Postal Code':'PostalCode'},inplace=True)

In [18]:
#we merge both dataframes using the colums PostalCode.
geo_ca_df = pd.merge(geo_df, ca_df, on='PostalCode')

In [19]:
#rearrange the dataframe to show the latitude and longitude as requested
geo_ca_df=geo_ca_df[['PostalCode','Borough','Neighborhood','Latitude','Longitude']]

In [20]:
geo_ca_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


## End of part 2


## Start Part 3 Explore and cluster the neighborhoods in Toronto

In [21]:
import numpy as np 
import pandas as pd 
import json
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
print('Libraries imported.')

Libraries imported.


## Focus in Toronto Borough

In [22]:
#Focus in Toronto Borough
Toronto_df= geo_ca_df[geo_ca_df['Borough'].str.contains('Toronto', na = False)].reset_index(drop=True)
Toronto_df.head()



,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


## Using the coordinates listed above, we create map of Toronto

In [23]:
#latitude and longitude from www.latlong.ne
latitude = 43.651070
longitude= -79.347015

# Using the coordinates listed above, we create map of Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map for each point inside Toronto_df (Data frame)
for lat, lng, label in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#737573',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Now we will introduce our credential for Foursquare and Define a function to obtain nearby venues in Foursquare (same as we used in the lab)

In [24]:
#Now we will introduce our credential for Foursquare
Limit = 100 # limit of number of venues returned by Foursquare API

CLIENT_ID = '4TVVK0SWNMD2CLIAS3HXSNXYZV4Z4PKZVLJNOK0BZ512X1TW' # your Foursquare ID
CLIENT_SECRET = 'NSGZJA213MM4QVCKFQNG5LGAQW5UJOTIOSCJBM3OSDNKGDDH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version, same as used in the lab

#Define a function to obtain nearby venues in Foursquare (same as we used in the lab)
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            Limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## use the function getNearbyVenues to obtain the venues and store it in a new data frame

In [ ]:
venues = getNearbyVenues(names=Toronto_df['Neighborhood'],
                                   latitudes=Toronto_df['Latitude'],
                                   longitudes=Toronto_df['Longitude'],
                                  )
print(venues.shape)
venues.head()



The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront


## Let's check how many venues were returned for each neighborhood


In [ ]:
venues.groupby('Neighborhood').count()

#### Let's find out how many unique categories can be curated from all the returned venues

In [ ]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

## Analyze Each Neighborhood as we did in the NewYork Lab

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()


In [ ]:
toronto_onehot.shape  #And let's examine the new dataframe size

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

#### Let's confirm the new size

In [ ]:
toronto_grouped.shape

#### Let's print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 10 clusters.

In [ ]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = Toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

Finally, let's visualize the resulting clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>